### External validation of clusters using housing data

In [140]:
# Import libraries
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Point
from shapely import wkt
import numpy as np
import shapely
from shapely.geometry import shape
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


### External Validation using housing listings data

In [142]:
# Read in labels saved from feature extraction
# Create an empty list to store the DataFrames
dataframes = []

# Loop through the range of 10 (0 to 9) to read in the CSV files
for i in range(10):
    filename = f'labels_df{i}.csv'  # Generate the file name
    df = pd.read_csv(filename, index_col=0)  # Read the CSV file into a DataFrame and set the first column as the index
    # Drop the first column (assuming it's the index column)
#    df = df.iloc[:, 1:]  # Assuming the first column is index and we drop it
    dataframes.append(df)



In [143]:
# Read in shapefile of Madrid with district boundaries
db = gpd.read_file('Distritos.shp')
# Set local crs in m 
db = db.to_crs(25830)

In [144]:
# Initialize grid_list
grid_list = []

# Set the initial CRS and step size
crs = '25830'
initial_step = 300

# Total bounds of the original GeoDataFrame (assuming it's called db)
a, b, c, d = db.total_bounds

# Create a grid for geometry with step size 300
gdf_grid = gpd.GeoDataFrame(
    geometry=[
        shapely.geometry.box(minx, miny, maxx, maxy)
        for minx, maxx in zip(np.arange(a, c, initial_step), np.arange(a, c, initial_step)[1:])
        for miny, maxy in zip(np.arange(b, d, initial_step), np.arange(b, d, initial_step)[1:])
    ],
    crs=crs,
).to_crs(db.crs)

# Append the grid to grid_list
grid_list.append(gdf_grid)


# Create grids to patch image
# Change the step size to change sizes
# Set the initial CRS and step size
crs = '25830'
initial_step = 500
final_step = 4000
step_increase = 400


# Loop through step sizes
for STEP in range(initial_step, final_step + 1, step_increase):
    # Total bounds of the original GeoDataFrame (assuming it's called db)
    a, b, c, d = db.total_bounds

    # Create a grid for geometry
    gdf_grid = gpd.GeoDataFrame(
        geometry=[
            shapely.geometry.box(minx, miny, maxx, maxy)
            for minx, maxx in zip(np.arange(a, c, STEP), np.arange(a, c, STEP)[1:])
            for miny, maxy in zip(np.arange(b, d, STEP), np.arange(b, d, STEP)[1:])
        ],
        crs=crs,
    ).to_crs(db.crs)

    # Append the current GeoDataFrame to the list
    grid_list.append(gdf_grid)


In [145]:
# Read in housing data
housep = pd.read_csv('Madrid_Sale.csv')
# Assuming 'long' and 'lat' are columns in your DataFrame 'housep'
housep['long'] = pd.to_numeric(housep['long'])
housep['lat'] = pd.to_numeric(housep['lat'])

# Create the 'geometry' column with Point geometries
housep['geometry'] = [Point(xy) for xy in zip(housep['long'], housep['lat'])]
# Create a GeoDataFrame
sale = gpd.GeoDataFrame(housep, crs="EPSG:4326", geometry=housep['geometry'])

In [146]:
# Drop duplicate listings with repeated ID
sale = sale.sort_values(by='period')  # Ensure the DataFrame is sorted by 'period' in ascending order
sale = sale.drop_duplicates(subset='ID', keep='last')

In [147]:
# change crs
sale = sale.to_crs('25830')

In [148]:
# add scale value to labels 
# Initialize the scale value for the first DataFrame
scale_value = 30

# Iterate over each DataFrame in the dataframes list
for idx, df in enumerate(dataframes):
    # Add a new column named 'scale' with the corresponding value
    df['scale'] = scale_value
    
    # Update the scale value for the next DataFrame
    if idx == 0:
        scale_value += 20  # Increase by 20 for the second DataFrame
    else:
        scale_value += 40  # Increase by 40 for each subsequent DataFrame


In [149]:
# Do spatial join between matching grid list and labels for each scale
# Create an empty list to store the merged DataFrames
merged_dfs = []

# Iterate over grid_list_dbs and dataframes simultaneously
for gdf_grid, df in zip(grid_list, dataframes):
    # Perform a spatial join using the index
    merged_df = pd.merge(gdf_grid, df,left_index=True, right_index=True, how='left')
    # Append the merged DataFrame to the list
    merged_dfs.append(merged_df)


In [151]:
# convert these merged_dfs to geodataframe

# Create an empty list to store the converted GeoDataFrames
converted_dfs = []

# Iterate over each merged DataFrame
for merged_df in merged_dfs:
    # Create a GeoDataFrame from the DataFrame
    converted_df = gpd.GeoDataFrame(merged_df, geometry='geometry')
    # Set the CRS to EPSG:25830
    converted_df.crs = 'EPSG:25830'
    # Append the converted GeoDataFrame to the list
    converted_dfs.append(converted_df)


In [152]:
# Do spatial join with housing data
# Create an empty list to store the spatially joined GeoDataFrames
spatial_joined_dfs = []

# Iterate over each converted GeoDataFrame
for converted_df in converted_dfs:
    # Perform a spatial join with the 'sale' GeoDataFrame
    spatial_joined_df = gpd.sjoin(converted_df, sale, how='inner', op='intersects')
    # Append the spatially joined GeoDataFrame to the list
    spatial_joined_dfs.append(spatial_joined_df)

# Now, spatial_joined_dfs contains the result of the spatial join of each item in converted_dfs with the 'sale' GeoDataFrame


/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed

In [186]:
# Directory to save the spatially joined GeoDataFrames
output_dir = "spatial_joined_data"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over each spatially joined GeoDataFrame
for idx, spatial_joined_df in enumerate(spatial_joined_dfs):
    # Construct the filename for saving
    filename = os.path.join(output_dir, f"spatial_joined_df_{idx}.csv")
    
    # Save the spatially joined GeoDataFrame to a CSV file
    spatial_joined_df.to_csv(filename, index=False)


In [174]:
# Calculate the std. dev in housing characteristics
# Assuming you have a list of dataframes named spatial_joined_dfs

# Initialize an empty list to store the dataframes
dfs_list = []

# Iterate over each DataFrame in spatial_joined_dfs
for idx, df in enumerate(spatial_joined_dfs):
    # Initialize a dictionary to store total standard deviations for each DataFrame
    total_std_devs = {}
    
    # Iterate over each column except the first (assuming the first column is 'geometry')
    for column in df.columns[1:17]:
        # Group the DataFrame by the current column
        grouped = df.groupby(column)
        
        # Calculate the standard deviation for 'price', 'area', 'conyr', 'maxfloor', 'dcount', 'dist_city_centre', and 'room' columns
        std_price = grouped['price'].std().sum()
        std_area = grouped['area'].std().sum()
        std_conyr = grouped['conyr'].std().sum()
        std_mxfloor = grouped['maxfloor'].std().sum()
        std_dcount = grouped['dcount'].std().sum()
        std_distc = grouped['dist_city_centre'].std().sum()
        std_room = grouped['room'].std().sum()

        
        # Store the total standard deviations in the dictionary
        total_std_devs[column] = {'std_price': std_price, 'std_area': std_area, 'std_conyr': std_conyr,
                                  'std_mxfloor': std_mxfloor, 'std_dcount': std_dcount, 'std_distc': std_distc,
                                  'std_room': std_room}  # Add 'std_scale'

    # Create a DataFrame from the dictionary
    df_temp = pd.DataFrame(total_std_devs).T.reset_index().rename(columns={'index': 'Column'})
    
    # Add 'DataFrame' column
    df_temp['DataFrame'] = f'DF_{idx}'
    
    # Append the DataFrame to the list
    dfs_list.append(df_temp)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs_list, ignore_index=True)


In [176]:
# Create number of feats variable 
# Extract the first number from the 'Column' column
df['num_feats'] = df['Column'].apply(lambda x: int(x.split('_')[0]))
# Extract the number of clusters from the 'Column' column
df['num_clus'] = df['Column'].apply(lambda x: int(x.split('_')[1]))

# Convert the 'num_feats' and 'num_clus' columns to integers
df['num_feats'] = df['num_feats'].astype(int)
df['num_clus'] = df['num_clus'].astype(int)


In [177]:
# Create a scale variable
# Define the mapping dictionary for scale values
scale_mapping = {
    'DF_0': 30,
    'DF_1': 50,
    'DF_2': 90,
    'DF_3': 130,
    'DF_4': 170,
    'DF_5': 210,
    'DF_6': 250,
    'DF_7': 290,
    'DF_8': 330,
    'DF_9': 370
}

# Map the 'DataFrame' column to scale values using the mapping dictionary
df['scale'] = df['DataFrame'].map(scale_mapping)


In [178]:
# Divide the standard deviation in 'price', 'area', and 'conyr' by the number of clusters to get the average std. dev per clusters
df['std_price_divided'] = df['std_price'] / df['num_clus']
df['std_area_divided'] = df['std_area'] / df['num_clus']
df['std_conyr_divided'] = df['std_conyr'] / df['num_clus']

In [180]:
df.to_csv('housing_scores2.csv')